In [3]:
import pandas as pd
import os
import csv

In [6]:
#raw input directory location
directory = './data/input_raw/'

# get a list of all the raw files in directory
file_list = os.listdir(directory)

In [10]:
df = pd.read_csv('../data/input_raw/DAY1.txt', sep='|', skiprows=2)
df.columns = ['Composite AxiomaID', 'Constituent AxiomaID', 'Weight']
df['Day'] = 1

df

,Composite AxiomaID,Constituent AxiomaID,Weight,Day
0,137M4ZH33,1XCHW5217,0.006269,1
1,137M4ZH33,2RG7PAW37,0.000388,1
2,137M4ZH33,2WGWDK1M4,0.007811,1
3,137M4ZH33,2WR7D8WX6,0.006882,1
4,137M4ZH33,2YPY4FLP8,0.006013,1
...,...,...,...,...
647183,ZZATSGMB8,ULBU2GAF2,0.018452,1
647184,ZZATSGMB8,VTDXP4DS7,0.027109,1
647185,ZZATSGMB8,WDYZG82Y3,0.033065,1
647186,ZZATSGMB8,ZNQZ9FKX1,0.017965,1


In [7]:

df = pd.DataFrame()

for file_name in file_list:
    file_directory = directory + file_name
    with open(file_directory, 'r') as f:
        date_line = f.readline().strip()
        extract_day = date_line.split('#DataDate: DAY')[1]
        
        columns_line = f.readline().strip()
        extract_columns = columns_line.split('#Columns: ')[1]
        columns_list = extract_columns.split('|')
        
        #skip first two rows
        file_df = pd.read_csv(directory+file_name, sep='|', skiprows=2)
        #drop last row
        file_df = file_df.iloc[:-1]
        #add the appropriate columns
        file_df.columns = columns_list
        #add the day as a row that pertains to the ETF data collected that day
        file_df['Day'] = int(extract_day)
        
        df = pd.concat([df, file_df])
        
print(df)
        
        
    

       Composite AxiomaID Constituent AxiomaID    Weight  Day
0               137M4ZH33            1XCHW5217  0.006269    1
1               137M4ZH33            2RG7PAW37  0.000388    1
2               137M4ZH33            2WGWDK1M4  0.007811    1
3               137M4ZH33            2WR7D8WX6  0.006882    1
4               137M4ZH33            2YPY4FLP8  0.006013    1
...                   ...                  ...       ...  ...
647295          ZZATSGMB8            TQX9AVVC8  0.022317    2
647296          ZZATSGMB8            ULBU2GAF2  0.018186    2
647297          ZZATSGMB8            VTDXP4DS7  0.027406    2
647298          ZZATSGMB8            WDYZG82Y3  0.033219    2
647299          ZZATSGMB8            ZNQZ9FKX1  0.018077    2

[1294487 rows x 4 columns]


In [84]:
day_list = df['Day'].unique()
day_list

array([1, 2])

In [88]:
filter_sum = df[df['Day'] == 1]
filter_sum = filter_sum[filter_sum['Composite AxiomaID'] == 'ZZATSGMB8']
filter_sum = filter_sum['Weight'].sum()
print(filter_sum)

2588748
1.0000023260000002


In [110]:
# For each DAY (DAY1 and DAY2), indicate how many distinct ETFs are present
distinct_etf_df = df[df['Day'] == day_list[0]]
distinct_etf_df = distinct_etf_df['Composite AxiomaID'].unique()
print(distinct_etf_df.size)

# second method with groupby
distinct_etf_df = df.groupby(['Day'])
distinct_etf_df = distinct_etf_df['Composite AxiomaID'].nunique()
print(distinct_etf_df)


2073
Day
1    2073
2    2073
Name: Composite AxiomaID, dtype: int64


In [111]:
# For each DAY, for each ETF provide a breakdown of how many constituents are present in each ETF

unique_constituent_df = df.groupby(['Day', 'Composite AxiomaID'])
unique_constituent_df = unique_constituent_df['Constituent AxiomaID'].nunique()
print(unique_constituent_df)


Day  Composite AxiomaID
1    137M4ZH33             127
     144DRUKM4              40
     14JUDRYZ8             496
     14N1R9B87             151
     14WBQZ7K6              93
                          ... 
2    ZXM8SKTF1              25
     ZXS3WWAK4             289
     ZY5UFBNY8             101
     ZYTH56V99              39
     ZZATSGMB8              31
Name: Constituent AxiomaID, Length: 4146, dtype: int64


In [144]:
# Compare DAY1 to DAY2.  For a given ETF, indicate which constituent has dropped from DAY1 to 
# DAY2, and which constituent has been added from DAY1 to DAY2

day1_dataframe = df[df['Day'] == 1]
day1_dataframe = day1_dataframe.drop(['Composite AxiomaID', 'Weight', 'Day'], axis = 1)
# day1_dataframe = day1_dataframe['Constituent AxiomaID'].unique()
# print(day1_dataframe)

day2_dataframe = df[df['Day'] == 2]
day2_dataframe = day2_dataframe.drop(['Composite AxiomaID', 'Weight', 'Day'], axis = 1)
# day2_dataframe = day2_dataframe['Constituent AxiomaID'].unique()
# print(day2_dataframe)
# print(day1_dataframe)
# print(day2_dataframe)
merge_dataframes = pd.merge(day1_dataframe, day2_dataframe, how='outer', indicator=True)
# in _merge 'left_only' should mean that day1 etf has not been carried over to day2
# in _merge 'left_only' should mean that this etf is an addition to day2
# in _merge 'both' should mean that this etf is a carry over

print(merge_dataframes)
print(merge_dataframes[merge_dataframes['_merge'] == 'left_only'])
print(merge_dataframes[merge_dataframes['_merge'] == 'right_only'])
print(merge_dataframes[merge_dataframes['_merge'] == 'both'])

         Constituent AxiomaID _merge
0                   1XCHW5217   both
1                   1XCHW5217   both
2                   1XCHW5217   both
3                   1XCHW5217   both
4                   1XCHW5217   both
...                       ...    ...
73774102            FPXH9M322   both
73774103            TPZ1MHMA7   both
73774104            ZAF45MZF2   both
73774105            X37R1RF14   both
73774106            S8AHBHL37   both

[73774107 rows x 2 columns]
Empty DataFrame
Columns: [Constituent AxiomaID, _merge]
Index: []
Empty DataFrame
Columns: [Constituent AxiomaID, _merge]
Index: []
         Constituent AxiomaID _merge
0                   1XCHW5217   both
1                   1XCHW5217   both
2                   1XCHW5217   both
3                   1XCHW5217   both
4                   1XCHW5217   both
...                       ...    ...
73774102            FPXH9M322   both
73774103            TPZ1MHMA7   both
73774104            ZAF45MZF2   both
73774105            X37R1

In [172]:
# For each ETF, indicate which constituent’s weight has changed the MOST from DAY1 to DAY2

# setting the decimal point to see the difference
pd.set_option('display.float_format', '{:.7f}'.format)

day1_dataframe = df[df['Day'] == 1]
day1_dataframe = day1_dataframe.rename(columns={'Weight' : 'Weight Day1'})
# print(day1_dataframe)

day2_dataframe = df[df['Day'] == 2]
day2_dataframe = day2_dataframe.rename(columns={'Weight' : 'Weight Day2'})
# print(day2_dataframe)

combined_dfs = pd.merge(day1_dataframe, day2_dataframe, on=['Composite AxiomaID','Constituent AxiomaID'], how='outer')
combined_dfs = combined_dfs.drop(['Day_x', 'Day_y'], axis=1)

combined_dfs['pct_change'] = (combined_dfs['Weight Day2'] - combined_dfs['Weight Day1']) / combined_dfs['Weight Day1']



print(combined_dfs)





       Composite AxiomaID Constituent AxiomaID  Weight Day1  Weight Day2  \
0               137M4ZH33            1XCHW5217    0.0062689    0.0062689   
1               137M4ZH33            2RG7PAW37    0.0003879    0.0003879   
2               137M4ZH33            2WGWDK1M4    0.0078106    0.0078106   
3               137M4ZH33            2WR7D8WX6    0.0068816    0.0068816   
4               137M4ZH33            2YPY4FLP8    0.0060129    0.0060129   
...                   ...                  ...          ...          ...   
647182          ZZATSGMB8            TQX9AVVC8    0.0222710    0.0222710   
647183          ZZATSGMB8            ULBU2GAF2    0.0184520    0.0184520   
647184          ZZATSGMB8            VTDXP4DS7    0.0271090    0.0271090   
647185          ZZATSGMB8            WDYZG82Y3    0.0330650    0.0330650   
647186          ZZATSGMB8            ZNQZ9FKX1    0.0179650    0.0179650   

        pct_change  
0        0.0000000  
1        0.0000000  
2        0.0000000  
3  